In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# carga de datos
gob_locales_mop = gpd.read_file('./gobiernos_locales_MOP/GobiernosLocales.shp')
gob_locales_total = pd.read_csv('./tablas_MOP/Munis Totales_Municipios.csv', encoding='utf-8', sep=";")
gob_locales_jujuy = pd.read_csv('./tablas_MOP/Municipios_Jujuy_NoSgo.csv', encoding='latin')

print("cantidad de gob locales en capa:", len(gob_locales_mop))
print("cantidad de gob locales en tabla:", len(gob_locales_total))
print("cantidad de gob locales de Jujuy:", len(gob_locales_jujuy))
print("cantidad de gob locales en tabla en CABA:", len(gob_locales_total.loc[gob_locales_total.Cpr == 2]))

cantidad de gob locales en capa: 2354
cantidad de gob locales en tabla: 2309
cantidad de gob locales de Jujuy: 3
cantidad de gob locales en tabla en CABA: 0


# Limpieza de datos en la tabla

In [3]:
# Se eliminan los registros con UTA2020 = 0 y los duplicados por index
gob_locales_total_clean = gob_locales_total.loc[gob_locales_total.UTA2020 != 0]
gob_locales_total_clean = gob_locales_total_clean.drop(994)
gob_locales_total_clean = gob_locales_total_clean.drop(1826)

In [4]:
lista_correcciones = [    
    # nombre, código UTA2020 correcto, (nombre al que corresponde), código en tabla
    ['Camilo Aldao', 'CAMILO ALDAO', '140630490', '140490', 'Caminiaga', '141543120'],
    ['Caminiaga', 'CAMINIAGA', '141543120', '143120', 'Canals', '141821624'],
    ['Canals', 'CANALS', '141821624', '141624', 'Candelaria Sud', '141683169'],
    ['Candelaria Sud', 'CANDELARIA SUD', '141683169', '143169', 'Cañada De Luque', '141681512'],
    ['Cañada De Luque', 'CAÑADA DE LUQUE', '141681512', '141512', 'Cañada De Machado', '141052616'],
    ['Cañada De Machado', 'CAÑADA DE MACHADO', '141052616', '142616', 'Cañada De Rio Pinto', '140492266'],
    ['Cañada De Rio Pinto', 'CAÑADA DE RIO PINTO', '140492266', '142266', 'Cañada Del Sauce', '140072021'], # corrección no detectada
]

# corrección de códigos erróneos
for i in lista_correcciones:
    gob_locales_total_clean.loc[gob_locales_total_clean.Mun ==  i[1], 'UTA2020'] = int(i[2])  # UTA2020 string - Int Ppo int
    gob_locales_total_clean.loc[gob_locales_total_clean.Mun ==  i[1], 'Codigo Ppo'] = int(i[2])
    gob_locales_total_clean.loc[gob_locales_total_clean.Mun ==  i[1], 'Int Ppo'] = int(i[2])
    gob_locales_total_clean.loc[gob_locales_total_clean.Mun ==  i[1], 'Cgl'] = int(i[3])

In [5]:
print("registros totales: ", len(gob_locales_total))
print("UTA2020 = 0: ", len(gob_locales_total.loc[gob_locales_total.UTA2020 == 0]))
print("UTA2020 duplicados: ", len(gob_locales_total.loc[gob_locales_total.duplicated('UTA2020'), 'UTA2020']))
print("UTA2020 totales: ", len(gob_locales_total)-len(gob_locales_total.loc[gob_locales_total.UTA2020 == 0])-len(gob_locales_total.loc[gob_locales_total.duplicated('UTA2020'), 'UTA2020']))
print("----------------------")
print("registros limpios totales: ", len(gob_locales_total_clean))

registros totales:  2309
UTA2020 = 0:  1
UTA2020 duplicados:  2
UTA2020 totales:  2306
----------------------
registros limpios totales:  2306


# Limpieza de la capa de Gobiernos Locales

In [6]:
# código ppo mayor a mil (elimina provincias y país)
gob_locales_mop_clean = gob_locales_mop.loc[gob_locales_mop.codigo_ppo > 1000]

In [7]:
# Busqueda de registros duplicados
gob_locales_mop_clean.loc[gob_locales_mop_clean.codigo_ppo.duplicated(), 'codigo_ppo']

2297    66160616.0
Name: codigo_ppo, dtype: float64

In [8]:
# Elimina registro duplicado
gob_locales_mop_clean = gob_locales_mop_clean.loc[(gob_locales_mop_clean.codigo_ppo != 66160616.0) | (gob_locales_mop_clean.observ == "Corregido por capa departamentos IGN")]

In [9]:
# intercambio de coordenadas
lista_geometrias = [
    [141192819, 140282196],
    [141261120, 140352252],
    [861336528, 860566405],
    [901190133, 900918560]
]

for p in lista_geometrias:

    geom_1 = list(gob_locales_mop_clean.loc[gob_locales_mop_clean.codigo_ppo == p[0], 'geometry'])[0]
    geom_2 = list(gob_locales_mop_clean.loc[gob_locales_mop_clean.codigo_ppo == p[1], 'geometry'])[0]

    gob_locales_mop_clean.loc[gob_locales_mop_clean.codigo_ppo == p[0], 'geometry'] = geom_2
    gob_locales_mop_clean.loc[gob_locales_mop_clean.codigo_ppo == p[1], 'geometry'] = geom_1

In [10]:
print("cantidad de gob locales en capa:", len(gob_locales_mop_clean))
print("cantidad de gob locales en capa en CABA:", len(gob_locales_mop_clean.loc[gob_locales_mop_clean.cpr == "02"]))
print("cantidad de gob locales en tabla:", len(gob_locales_total_clean))
print("cantidad de gob locales en tabla en CABA:", len(gob_locales_total_clean.loc[gob_locales_total_clean.Cpr == 2]))

cantidad de gob locales en capa: 2328
cantidad de gob locales en capa en CABA: 15
cantidad de gob locales en tabla: 2306
cantidad de gob locales en tabla en CABA: 0


# Cruce entre gobiernos locales y tabla

In [11]:
# merge entre la tabla de gob locales y la capa geográfica
gob_merge = gob_locales_mop_clean.merge(gob_locales_total_clean, left_on='codigo_ppo', right_on='UTA2020', how='left')

In [12]:
# separación entre los gob locales que están en la capa y la tabla de los que no están en la tabla
gob_merge_sinMatch = gob_merge.loc[(gob_merge.UTA2020.isnull()) & (gob_merge.codigo_ppo > 30000000)]
gob_merge_conMatch = gob_merge.loc[(gob_merge.UTA2020.notnull()) | (gob_merge.codigo_ppo < 30000000)]

In [13]:
print("cantidad de gob locales (capa):", len(gob_merge_sinMatch)+len(gob_merge_conMatch))
print("cantidad de gob locales sin matcheo:", len(gob_merge_sinMatch), "("+ str(len(gob_merge_sinMatch.loc[gob_merge_sinMatch.cpr == "02"])) +" CABA)")
print("cantidad de gob locales con matcheo:", len(gob_merge_conMatch), "("+ str(len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr != "02"]))+" + "+str(len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr == "02"]))+" CABA)")
print("----------------------")
print("cantidad de gob locales nuevos de Jujuy:", len(gob_locales_jujuy))
print("----------------------")
print("cantidad total de gob locales:", len(gob_locales_jujuy)+len(gob_merge_conMatch), "("+ str(len(gob_locales_jujuy)+len(gob_merge_conMatch)-len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr == "02"]))+" sin CABA)")

cantidad de gob locales (capa): 2328
cantidad de gob locales sin matcheo: 16 (0 CABA)
cantidad de gob locales con matcheo: 2312 (2297 + 15 CABA)
----------------------
cantidad de gob locales nuevos de Jujuy: 3
----------------------
cantidad total de gob locales: 2315 (2300 sin CABA)


## Procesar registros de tabla sin cruzar

In [14]:
# generación de un campo booleano que guarda si el gob local está presente en la capa
gob_locales_total_clean['isinmatch'] = 0

for i in list(gob_locales_total_clean.UTA2020):
    if len(gob_merge_conMatch.loc[gob_merge_conMatch.codigo_ppo == i]) == 0:
        gob_locales_total_clean.loc[gob_locales_total_clean.UTA2020 == i, 'isinmatch'] = 1
# dataframe con gob locales en tabla sin matchear con capa de gob locales
gob_locales_total_clean_nomatch = gob_locales_total_clean.loc[gob_locales_total_clean.isinmatch == 1]

In [15]:
print("CAPA")
print("cantidad de gob locales:", len(gob_merge_sinMatch)+len(gob_merge_conMatch))
print("cantidad de gob locales sin matcheo:", len(gob_merge_sinMatch), "("+ str(len(gob_merge_sinMatch.loc[gob_merge_sinMatch.cpr == "02"])) +" CABA)")
print("cantidad de gob locales con matcheo:", len(gob_merge_conMatch), "("+ str(len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr != "02"]))+" + "+str(len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr == "02"]))+" CABA)")
print("----------------------")
print("JUJUY")
print("cantidad de gob locales nuevos de Jujuy:", len(gob_locales_jujuy))
print("----------------------")
print("TABLA")
print("cantidad de registros totales:", len(gob_locales_total))
print("cantidad de registros UTA2020=0:", len(gob_locales_total.loc[gob_locales_total.UTA2020 == 0]))
print("cantidad de registros UTA2020 duplicados: ", len(gob_locales_total.loc[gob_locales_total.duplicated('UTA2020'), 'UTA2020']))
print("cantidad de gob locales en tabla:", len(gob_locales_total_clean))
print("cantidad de gob locales en tabla no matcheados:", len(gob_locales_total_clean_nomatch))
print("----------------------")
print("TOTAL")
totalgobs=len(gob_locales_jujuy)+len(gob_merge_conMatch)+len(gob_locales_total_clean_nomatch)
print("cantidad total de gob locales:", str(totalgobs), "("+ str(totalgobs-len(gob_merge_conMatch.loc[gob_merge_conMatch.cpr == "02"]))+" sin CABA)")


CAPA
cantidad de gob locales: 2328
cantidad de gob locales sin matcheo: 16 (0 CABA)
cantidad de gob locales con matcheo: 2312 (2297 + 15 CABA)
----------------------
JUJUY
cantidad de gob locales nuevos de Jujuy: 3
----------------------
TABLA
cantidad de registros totales: 2309
cantidad de registros UTA2020=0: 1
cantidad de registros UTA2020 duplicados:  2
cantidad de gob locales en tabla: 2306
cantidad de gob locales en tabla no matcheados: 9
----------------------
TOTAL
cantidad total de gob locales: 2324 (2309 sin CABA)


In [16]:
# campos longitud y latitud a float en gob locales no matcheados
gob_locales_total_clean_nomatch['Longitud'] = gob_locales_total_clean_nomatch['Longitud'].str.replace(',', '.')
gob_locales_total_clean_nomatch['Latitud'] = gob_locales_total_clean_nomatch['Latitud'].str.replace(',', '.')
gob_locales_total_clean_nomatch['Longitud'] = gob_locales_total_clean_nomatch.Longitud.astype(float)
gob_locales_total_clean_nomatch['Latitud'] = gob_locales_total_clean_nomatch.Latitud.astype(float)

# creación de un dataframe a partir de los campos de coordenadas para los gob locales no matcheados
gob_locales_total_clean_nomatch = gpd.GeoDataFrame(
    gob_locales_total_clean_nomatch, 
    geometry= gpd.points_from_xy(gob_locales_total_clean_nomatch.Longitud, gob_locales_total_clean_nomatch.Latitud))

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [17]:
# creación de una geometría de polígono a partir de un buffer de 10cm a partir de las coordenadas para los gob locales sin matchear
gob_locales_total_clean_nomatch = gob_locales_total_clean_nomatch.set_crs("EPSG:4326")
gob_locales_total_clean_nomatch = gob_locales_total_clean_nomatch.to_crs("EPSG:3857")
gob_locales_total_clean_nomatch['geometry'] = gob_locales_total_clean_nomatch.buffer(0.01)
gob_locales_total_clean_nomatch = gob_locales_total_clean_nomatch.to_crs("EPSG:4326")

In [18]:
# edición del campo gna
gob_locales_total_clean_nomatch.loc[gob_locales_total_clean_nomatch.tipgobloc == 'Municipio', 'Gna'] = 'Municipio'
gob_locales_total_clean_nomatch.loc[gob_locales_total_clean_nomatch.tipgobloc == 'Municipio de Primera', 'Gna'] = 'Municipio'
gob_locales_total_clean_nomatch.loc[gob_locales_total_clean_nomatch.tipgobloc == 'Comuna', 'Gna'] = 'Comuna'
gob_locales_total_clean_nomatch.loc[gob_locales_total_clean_nomatch.tipgobloc == 'Comision Municipal B', 'Gna'] = 'Comision Municipal'

## Procesar polígonos de Jujuy

In [19]:
# creación de un dataframe a partir de los campos de coordenadas para los gob locales de Jujuy
gob_locales_jujuy = gpd.GeoDataFrame(
    gob_locales_jujuy, 
    geometry= gpd.points_from_xy(gob_locales_jujuy.y, gob_locales_jujuy.x))

# generación del campo UTA2020
gob_locales_jujuy['UTA2020'] = gob_locales_jujuy.apply(lambda row: str(row['ID_Depto_INDEC']) + str(row['ID_GL_INDEC_2020 (sin reso)'])[2:], axis=1)

In [20]:
# filtro de las columnas de interés y cambio de nombre 
gob_locales_jujuy_2 = gob_locales_jujuy[['Provincia', 'UTA2020', 'Departamento', 'ID_GL_INDEC_2020 (sin reso)', 'Nombre del Gobierno Local', 'Categoría de Gobierno', 'x', 'y', 'geometry']]
gob_locales_jujuy_2.columns = ['Provincia', 'UTA2020', 'Departamento', 'In1', 'Mun', 'tipgobloc', 'Latitud', 'Longitud', 'geometry']
# creación de una geometría de polígono a partir de un buffer de 10cm a partir de las coordenadas para los gob locales de Jujuy
gob_locales_jujuy_2 = gob_locales_jujuy_2.set_crs("EPSG:4326")
gob_locales_jujuy_2 = gob_locales_jujuy_2.to_crs("EPSG:3857")
gob_locales_jujuy_2['geometry'] = gob_locales_jujuy_2.buffer(0.01)
gob_locales_jujuy_2 = gob_locales_jujuy_2.to_crs("EPSG:4326")
# edición del campo gna
gob_locales_jujuy_2['Gna'] = 'Comisión Municipal'
gob_locales_jujuy_2

,Provincia,UTA2020,Departamento,In1,Mun,tipgobloc,Latitud,Longitud,geometry,Gna
0,Jujuy,380286064,Humahuaca,386064,Palca de Aparzo,Comisión Municipal,-23.098639,-65.108196,"POLYGON ((-65.10820 -23.09864, -65.10820 -23.0...",Comisión Municipal
1,Jujuy,380286065,Humahuaca,386065,Uquia,Comisión Municipal,-23.304496,-65.357177,"POLYGON ((-65.35718 -23.30450, -65.35718 -23.3...",Comisión Municipal
2,Jujuy,380986204,Tumbaya,386204,El Moreno,Comisión Municipal,-23.853668,-65.831784,"POLYGON ((-65.83178 -23.85367, -65.83178 -23.8...",Comisión Municipal


## Concatenar y exportar datos

In [21]:
# CABA fix
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'Provincia'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'provincia']
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'Mun'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'nam']
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'Gna'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'gna']
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'tipgobloc'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'gna']
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'Fdc'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'fdc']
gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'Fuente'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'fuente']

gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'UTA2020'] = gob_merge_conMatch.loc[gob_merge_conMatch.UTA2020.isnull(), 'codigo_ppo']

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [22]:
# generación de un booleano para reconocer los gob locales cuya geometría se generó a partir de un buffer
gob_merge_conMatch['buffer'] = 0
gob_locales_total_clean_nomatch['buffer'] = 1
gob_locales_jujuy_2['buffer'] = 1

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [23]:
# concatenación de todos los dataframes de gob locales
df_final = pd.concat([gob_merge_conMatch, gob_locales_total_clean_nomatch, gob_locales_jujuy_2])
# edición de los campos UTA2020, indec2019 y CodRedatam
df_final['UTA2020'] = df_final.apply(lambda row: str(row['UTA2020']).split('.')[0], axis=1)
df_final['UTA2020'] = df_final.apply(lambda row: '0'+row['UTA2020'] if len(row['UTA2020']) ==8 else row['UTA2020'], axis=1)
df_final['indec2019'] = df_final.apply(lambda row: str(row['indec2019']).split('.')[0], axis=1)
df_final['CodRedatam'] = df_final.apply(lambda row: str(row['CodRedatam']).split('.')[0], axis=1)
# generación de los códigos de pcia, depto y gob local
df_final['cpr'] = df_final.apply(lambda row: str(row['UTA2020'])[:2], axis=1)
df_final['cde'] = df_final.apply(lambda row: str(row['UTA2020'])[2:5], axis=1)
df_final['cgl'] = df_final.apply(lambda row: str(row['UTA2020'])[:2] + str(row['UTA2020'])[5:9], axis=1)
# capos strign a formato Title
df_final['Provincia'] = df_final.Provincia.str.title()
df_final['Departamento'] = df_final.Departamento.str.title()
df_final['Mun'] = df_final.Mun.str.title()
# filtro de los atributos de interés
df_final = df_final[['cpr', 'Provincia', 'cde', 'Departamento', 'cgl', 'Mun', 'UTA2020', 'indec2019', 'in1', 
'CodRedatam', 'Gna', 'tipgobloc', 'Fdc', 'Fuente', 'Latitud', 'Longitud', 'buffer', 'geometry']]
# cambio de nombre de columna
df_final.rename(columns={'Mun':'Municipio'}, inplace=True)
# ordenar por UTA2020
df_final.sort_values('UTA2020', inplace=True)
df_final.reset_index(inplace=True, drop=True)

print(len(df_final) == len(df_final.UTA2020.unique()))
df_final.head()

True


,cpr,Provincia,cde,Departamento,cgl,Municipio,UTA2020,indec2019,in1,CodRedatam,Gna,tipgobloc,Fdc,Fuente,Latitud,Longitud,buffer,geometry
0,02,C.A.B.A.,007,NaN,022007,Comuna 1,020072007,nan,020001,nan,Comuna,Comuna,None,Capa Departamentos IGN,NaN,NaN,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5..."
1,02,C.A.B.A.,014,NaN,022014,Comuna 2,020142014,nan,020002,nan,Comuna,Comuna,None,Capa Departamentos IGN,NaN,NaN,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5..."
2,02,C.A.B.A.,021,NaN,022021,Comuna 3,020212021,nan,020003,nan,Comuna,Comuna,None,Capa Departamentos IGN,NaN,NaN,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5..."
3,02,C.A.B.A.,028,NaN,022028,Comuna 4,020282028,nan,020004,nan,Comuna,Comuna,None,Capa Departamentos IGN,NaN,NaN,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6..."
4,02,C.A.B.A.,035,NaN,022035,Comuna 5,020352035,nan,020005,nan,Comuna,Comuna,None,Capa Departamentos IGN,NaN,NaN,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5..."


In [24]:
# exportación de los gob locales
# gob_merge_sinMatch.to_file('./gobiernos_locales_shp/GobiernosLocales_sinMatch.shp')
df_final.to_file('./gobiernos_locales_shp/GobiernosLocales.shp')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


# Agregar gob locales faltantes

In [25]:
# Se recuperan dos gob locales faltantes de la capa geográfica original, que fueron filtrados por tener datos incompletos
df_2faltantes = gob_locales_mop.loc[(gob_locales_mop.codigo_ppo == 140072021)|(gob_locales_mop.codigo_ppo == 380776140), ['cpr', 'provincia',  'cgl', 'nam', 'codigo_ppo', 'gna', 'fdc', 'fuente', 'geometry']]
df_2faltantes.rename(columns={'provincia':'Provincia', 'nam':'Municipio', 'codigo_ppo':'UTA2020', 'gna':'Gna', 'fdc':'Fdc', 'fuente':'Fuente'}, inplace=True)
# se cargan los datos de los gob locales
df_2faltantes.loc[df_2faltantes.UTA2020 == 140072021, 'cde'] = '007'
df_2faltantes.loc[df_2faltantes.UTA2020 == 140072021, 'Departamento'] = 'Calamuchita'
df_2faltantes.loc[df_2faltantes.UTA2020 == 380776140, 'cde'] = '077'
df_2faltantes.loc[df_2faltantes.UTA2020 == 380776140, 'Departamento'] = 'Santa Catalina'
df_2faltantes['indec2019'] = 0
df_2faltantes['in1'] = 0
df_2faltantes['CodRedatam'] = 0
df_2faltantes['tipgobloc'] = df_2faltantes['Gna']
df_2faltantes['Latitud'] = df_2faltantes.centroid.y
df_2faltantes['Longitud'] = df_2faltantes.centroid.x
df_2faltantes['buffer'] = 0
df_2faltantes.UTA2020 = df_2faltantes.UTA2020.astype(int)
df_2faltantes.UTA2020 = df_2faltantes.UTA2020.astype(str)
df_2faltantes

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  del sys.path[0]
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,cpr,Provincia,cgl,Municipio,UTA2020,Gna,Fdc,Fuente,geometry,cde,Departamento,indec2019,in1,CodRedatam,tipgobloc,Latitud,Longitud,buffer
152,38,Jujuy,386140,Cusi Cusi,380776140,Comisión Municipal,Direc. Grl. de Catastro,IGN: Capa municipios,"POLYGON ((-66.40999 -22.18012, -66.40951 -22.1...",077,Santa Catalina,0,0,0,Comisión Municipal,-22.492077,-66.741029,0
498,14,Córdoba,142021,Cañada del Sauce,140072021,Comuna,Direc. Grl. de Catastro,IGN: Capa municipios,"MULTIPOLYGON (((-64.62405 -32.35469, -64.62334...",007,Calamuchita,0,0,0,Comuna,-32.382720,-64.636927,0


In [26]:
# se unen estos dos gob locales al resto
df_merged = pd.concat([df_final, df_2faltantes]).reset_index(drop=True)
df_merged = df_merged.sort_values(by='UTA2020').reset_index(drop=True)
df_merged.loc[df_merged.Latitud.isnull(), 'Latitud'] = df_merged.loc[df_merged.Latitud.isnull()].centroid.y
df_merged.loc[df_merged.Longitud.isnull(), 'Longitud'] = df_merged.loc[df_merged.Longitud.isnull()].centroid.x
df_merged.loc[df_merged.Provincia == "Córdoba", 'Provincia'] = "Cordoba"
df_merged.head()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """


,cpr,Provincia,cde,Departamento,cgl,Municipio,UTA2020,indec2019,in1,CodRedatam,Gna,tipgobloc,Fdc,Fuente,Latitud,Longitud,buffer,geometry
0,02,C.A.B.A.,007,NaN,022007,Comuna 1,020072007,nan,020001,nan,Comuna,Comuna,None,Capa Departamentos IGN,-34.6064,-58.3715,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5..."
1,02,C.A.B.A.,014,NaN,022014,Comuna 2,020142014,nan,020002,nan,Comuna,Comuna,None,Capa Departamentos IGN,-34.5858,-58.3949,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5..."
2,02,C.A.B.A.,021,NaN,022021,Comuna 3,020212021,nan,020003,nan,Comuna,Comuna,None,Capa Departamentos IGN,-34.6138,-58.4027,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5..."
3,02,C.A.B.A.,028,NaN,022028,Comuna 4,020282028,nan,020004,nan,Comuna,Comuna,None,Capa Departamentos IGN,-34.6421,-58.3875,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6..."
4,02,C.A.B.A.,035,NaN,022035,Comuna 5,020352035,nan,020005,nan,Comuna,Comuna,None,Capa Departamentos IGN,-34.6174,-58.4206,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5..."


## Completar los datos CodRedatam de CABA

In [30]:
df_merged.loc[df_merged.cpr == '02', 'CodRedatam'] = ("2"+ df_merged.loc[df_merged.cpr == '02', 'in1'].str[-3:] + "0001")
df_merged.loc[df_merged.cpr == '02', 'Departamento'] = df_merged.loc[df_merged.cpr == '02', 'Municipio']
df_merged.head()

,cpr,Provincia,cde,Departamento,cgl,Municipio,UTA2020,indec2019,in1,CodRedatam,Gna,tipgobloc,Fdc,Fuente,Latitud,Longitud,buffer,geometry
0,02,C.A.B.A.,007,Comuna 1,022007,Comuna 1,020072007,nan,020001,20010001,Comuna,Comuna,None,Capa Departamentos IGN,-34.6064,-58.3715,0,"POLYGON ((-58.38609 -34.57823, -58.38592 -34.5..."
1,02,C.A.B.A.,014,Comuna 2,022014,Comuna 2,020142014,nan,020002,20020001,Comuna,Comuna,None,Capa Departamentos IGN,-34.5858,-58.3949,0,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.5..."
2,02,C.A.B.A.,021,Comuna 3,022021,Comuna 3,020212021,nan,020003,20030001,Comuna,Comuna,None,Capa Departamentos IGN,-34.6138,-58.4027,0,"POLYGON ((-58.41026 -34.59811, -58.40939 -34.5..."
3,02,C.A.B.A.,028,Comuna 4,022028,Comuna 4,020282028,nan,020004,20040001,Comuna,Comuna,None,Capa Departamentos IGN,-34.6421,-58.3875,0,"POLYGON ((-58.35517 -34.61945, -58.35511 -34.6..."
4,02,C.A.B.A.,035,Comuna 5,022035,Comuna 5,020352035,nan,020005,20050001,Comuna,Comuna,None,Capa Departamentos IGN,-34.6174,-58.4206,0,"POLYGON ((-58.42271 -34.59773, -58.42174 -34.5..."


In [31]:
df_merged.to_file('./gobiernos_locales_shp/GobiernosLocales.shp')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


# Chequeos

## Deptos mal

In [32]:
df_deptos = gpd.read_file('./departamentos/departamento.shp')
df_gob_locales= gpd.read_file('./gobiernos_locales_shp/GobiernosLocales.shp')
df_gob_locales_check = df_gob_locales.copy()
df_gob_locales_check.geometry = df_gob_locales.centroid

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.


### Chequeo Espacial

In [33]:
# se ajustan los nombres de la capa de gob locales para que coincidan con la de departamentos
df_gob_loc = df_gob_locales_check[['Provincia', 'Departamen', 'Municipio', 'UTA2020', 'geometry']].sjoin(df_deptos[['nam', 'geometry']])
df_gob_loc.Departamen = df_gob_loc.Departamen.str.upper()
df_gob_loc.nam = df_gob_loc.nam.str.upper()
df_gob_loc.nam = df_gob_loc.nam.str.replace('Á','A')
df_gob_loc.nam = df_gob_loc.nam.str.replace('É','E')
df_gob_loc.nam = df_gob_loc.nam.str.replace('Í','I')
df_gob_loc.nam = df_gob_loc.nam.str.replace('Ó','O')
df_gob_loc.nam = df_gob_loc.nam.str.replace('Ú','U')
df_gob_loc.nam = df_gob_loc.nam.str.replace('JUAN BAUTISTA ALBERDI','JUAN B ALBERDI')
df_gob_loc.nam = df_gob_loc.nam.str.replace('JOSE C. PAZ','JOSE C PAZ')
df_gob_loc.nam = df_gob_loc.nam.str.replace('LEANDRO N. ALEM','LEANDRO N ALEM')
df_gob_loc.Departamen = df_gob_loc.Departamen.str.replace('LEANDRO N. ALEM','LEANDRO N ALEM')
df_gob_loc.nam = df_gob_loc.nam.str.replace('CORONEL DE MARINA LEONARDO ROSALES','CORONEL DE MARINA L ROSALES')
df_gob_loc.nam = df_gob_loc.nam.str.replace('JUAN FELIPE IBARRA','JUAN F IBARRA')
df_gob_loc.nam = df_gob_loc.nam.str.replace('GENERAL GÜEMES','GENERAL GUEMES')
df_gob_loc.nam = df_gob_loc.nam.str.replace('MAYOR LUIS J. FONTANA','MAYOR LUIS J FONTANA')
df_gob_loc.nam = df_gob_loc.nam.str.replace("O'HIGGINS",'O HIGGINS')
df_gob_loc.nam = df_gob_loc.nam.str.replace('DR. MANUEL BELGRANO', 'DOCTOR MANUEL BELGRANO')
df_gob_loc.nam = df_gob_loc.nam.str.replace('GENERAL FELIPE VARELA','CORONEL FELIPE VARELA')
df_gob_loc.nam = df_gob_loc.nam.str.replace('ANGEL VICENTE PEÑALOZA','GENERAL ANGEL VERA PEÑALOZA')
df_gob_loc.nam = df_gob_loc.nam.str.replace('GENERAL JUAN FACUNDO QUIROGA','GENERAL JUAN F QUIROGA')
df_gob_loc.nam = df_gob_loc.nam.str.replace('GENERAL ORTIZ DE OCAMPO','GENERAL OCAMPO')
df_gob_loc.nam = df_gob_loc.nam.str.replace('MALARGÜE','MALARGUE')
df_gob_loc.nam = df_gob_loc.nam.str.replace('GÜER AIKE','GUER AIKE')

In [34]:
# se chequea que no hay gob locales con el nombre de departamento no coincidente con la capa geográfica
# se filtran algunos casos que fueron chequeados en el mapa
print("Cantidad de gob locales con deptos incorrectos", len(df_gob_loc.loc[(df_gob_loc.Departamen != df_gob_loc.nam)&(df_gob_loc.Departamen.notnull())
    &(df_gob_loc.Provincia != 'Salta')&(df_gob_loc.Provincia != 'Santa Fe')&(df_gob_loc.Provincia != 'Rio Negro')&(df_gob_loc.Provincia != 'Buenos Aires')
    &(df_gob_loc.Provincia != 'Catamarca')&(df_gob_loc.Provincia != 'Chaco')&(df_gob_loc.Provincia != 'Jujuy')&(df_gob_loc.Provincia != 'La Pampa')
    &(df_gob_loc.Departamen != 'JIMENEZ')&(df_gob_loc.Departamen != 'BELGRANO')
    &(df_gob_loc.UTA2020 != '141471400')&(df_gob_loc.UTA2020 != '140070021')&(df_gob_loc.UTA2020 != '141823232')
    &(df_gob_loc.UTA2020 != '141681526')&(df_gob_loc.UTA2020 != '141122735')])  )

Cantidad de gob locales con deptos incorrectos 0


### Chequeo de Códigos

In [35]:
# se chequea que los códigos de los deptos se correspondan con un único nombre de depto, para cada provincia
df_deptos_unique = df_gob_locales_check.loc[(~df_gob_locales_check.duplicated(subset=['cpr', 'cde', 'Departamen'])) & (df_gob_locales_check.cpr != '02')] \
    .groupby(['cpr', 'cde'])\
    .size()\
    .reset_index()
df_deptos_unique.rename(columns={df_deptos_unique.columns[2]:'cde_count'}, inplace=True)
df_deptos_unique_dup = df_deptos_unique.loc[df_deptos_unique.cde_count > 1].reset_index(drop=True)
df_deptos_unique_dup

,cpr,cde,cde_count
0,74,035,2
1,74,042,2
2,74,056,2
3,82,014,2
4,82,084,2


In [36]:
# se realiza un conteo de deptos para capa código
df_deptos_count = df_gob_locales_check.loc[(df_gob_locales_check.cpr != '02')] \
    .groupby(['cpr', 'cde', 'Departamen'])\
    .size()\
    .reset_index()
# lista para corregir nombres
lista_map_deptos = []

for i in range(len(df_deptos_unique_dup)):
    # en función de la cantidad de nombres para ese código, se determina cual es el correcto
    depto_correcto = df_deptos_count.loc[(df_deptos_count.cpr == df_deptos_unique_dup.loc[i, 'cpr']) & (df_deptos_count.cde == df_deptos_unique_dup.loc[i, 'cde'])]\
        .rename(columns={0:'count'})\
        .sort_values(by='count', ascending=False)\
        .reset_index(drop=True)\
        .loc[0, 'Departamen']
    # se filtra el gob local a corregir 
    df_goblocal_fix_depto = df_gob_locales_check.loc[(df_gob_locales_check.cpr == df_deptos_unique_dup.loc[i, 'cpr']) & (df_gob_locales_check.cde == df_deptos_unique_dup.loc[i, 'cde'])
                                                 & (~df_gob_locales_check.duplicated(subset=['cpr', 'cde', 'Departamen'])), ['cpr', 'cde', 'Departamen', 'Municipio']]
    print(df_goblocal_fix_depto)
    print("Correcto:", depto_correcto)
    print("----------------------")

    # se agregan a la lista los datos del gob local a corregir
    lista_map_deptos.append([
        list(df_goblocal_fix_depto.loc[df_goblocal_fix_depto.Departamen != depto_correcto].index)[0],
        list(df_goblocal_fix_depto.loc[df_goblocal_fix_depto.Departamen != depto_correcto, 'cpr'])[0], 
        list(df_goblocal_fix_depto.loc[df_goblocal_fix_depto.Departamen != depto_correcto, 'cde'])[0],
        depto_correcto,
        list(df_goblocal_fix_depto.loc[df_goblocal_fix_depto.Departamen != depto_correcto, 'Municipio'])[0],
        list(df_goblocal_fix_depto.loc[df_goblocal_fix_depto.Departamen != depto_correcto, 'Departamen'])[0],
    ])

print(lista_map_deptos)

     cpr  cde         Departamen     Municipio
1619  74  035  General Pedernera  Justo Daract
1623  74  035   Coronel Pringles  Juan Llerena
Correcto: General Pedernera
----------------------
     cpr  cde                 Departamen        Municipio
1626  74  042           Gobernador Dupuy  Buena Esperanza
1634  74  042  Juan Martin De Pueyrredon            Navia
Correcto: Gobernador Dupuy
----------------------
     cpr  cde                 Departamen    Municipio
1642  74  056  Juan Martin De Pueyrredon  Desaguadero
1648  74  056          General Pedernera  Alto Pelado
Correcto: Juan Martin De Pueyrredon
----------------------
     cpr  cde     Departamen        Municipio
1687  82  014        Caseros          Casilda
1700  82  014  General Lopez  Cañada Del Ucle
Correcto: Caseros
----------------------
     cpr  cde   Departamen       Municipio
1899  82  084      Rosario     Arroyo Seco
1923  82  084  San Lorenzo  Coronel Arnold
Correcto: Rosario
----------------------
[[1623, '74', 

In [37]:
for i in lista_map_deptos:
    # nombre del depto previo a la corrección
    depto_antes = list(df_gob_locales.loc[(df_gob_locales.cpr == i[1]) & (df_gob_locales.cde == i[2])
                             & (df_gob_locales.Municipio == i[4]), 'Departamen'])[0]
    if i[5] == depto_antes:
        # corrección del nombre de depto
        df_gob_locales.loc[(df_gob_locales.cpr == i[1]) & (df_gob_locales.cde == i[2])
                                & (df_gob_locales.Municipio == i[4]), ['Departamen']] = i[3]
        # nombre del depto luego de la corrección
        depto_despues = list(df_gob_locales.loc[(df_gob_locales.cpr == i[1]) & (df_gob_locales.cde == i[2])
                                & (df_gob_locales.Municipio == i[4]), 'Departamen'])[0]
        print(depto_antes, ' - ', depto_despues)

Coronel Pringles  -  General Pedernera
Juan Martin De Pueyrredon  -  Gobernador Dupuy
General Pedernera  -  Juan Martin De Pueyrredon
General Lopez  -  Caseros
San Lorenzo  -  Rosario


In [38]:
df_gob_locales.to_file('./gobiernos_locales_shp/GobiernosLocales.shp')